# Install Py3Dmol and BioPython

In [ ]:
#Installing py3Dmol and biopython
!pip install py3Dmol biopython

#Importing py3Dmol for safety
import py3Dmol

# Install GROMACS (for the first time)



1.  Change the "**Runtime**" of the Google Colab and enable "**T4 GPU**" under "Hardware accelerator" section
2.  **Mount Google Drive** so that the work will be automatically saved in the Drive (as backup purpose) and **install GROMACS using following commands**



In [ ]:
#1. Mount Google Drive:
## the process will ask for the permission. allow it.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#2. create a folder called "gromacs" and then install GROMACS inside that folder
%cd /content/drive/MyDrive/ #or your chosen file_path
!mkdir gromacsTools
%cd gromacsTools

#3. install gromacs

## Download and unzip the compressed folder of GROMACS 2020.6 version
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs.tar.gz
!tar xzf gromacs.tar.gz

## It is recommended to upgrade cmake
!apt-get update
!pip install cmake --upgrade

## download the libhwloc5 package and install the package using dpkg:
!wget http://archive.ubuntu.com/ubuntu/pool/universe/h/hwloc/libhwloc5_1.11.9-1_amd64.deb
!dpkg -i libhwloc5_1.11.9-1_amd64.deb

# To use already installed GROMACS

If you already have installed GROMACS in the Google Drive following the above instruction during any previous runtime (inside /MyDrive/gromacsTools/), you can start using GROMACS from here.

In [ ]:
# Mount Google Drive:
## the process will ask for the permission. allow it.
from google.colab import drive
drive.mount('/content/drive') #or your chosen file_path

In [ ]:
!chmod +x /content/drive/MyDrive/gromacsTools/gromacs/bin/gmx

## download the libhwloc5 package and install the package using dpkg:
!wget http://archive.ubuntu.com/ubuntu/pool/universe/h/hwloc/libhwloc5_1.11.9-1_amd64.deb
!dpkg -i libhwloc5_1.11.9-1_amd64.deb

# check if GROMACS is working properly or not
!ln -sf /content/drive/MyDrive/gromacsTools/gromacs/bin/gmx /usr/bin/
!gmx --version

In [ ]:
!apt-get update
!apt-get install -y nvidia-cuda-toolkit

# Energy Minimization in Molecular Dynamics

## Theory
Energy minimization (EM) is a crucial first step in MD simulations that:
1. Removes bad contacts or overlaps between atoms
2. Relaxes the system to its local energy minimum
3. Prevents system instability during dynamics

## Input Files Required
1. **step3_input.gro**: Coordinate file containing:
   - Atomic positions
   - Box vectors
   - System topology

2. **step4.0_minimization.mdp**: Parameter file specifying:
   - Minimization algorithm (e.g., steepest descent)
   - Maximum steps
   - Energy step size
   - Convergence criteria

3. **topol.top**: Topology file containing:
   - Molecular topology
   - Force field parameters
   - Bond definitions
   - Angle definitions

4. **index.ndx**: Index file defining:
   - Atom groups
   - Selections for analysis

## Output Analysis
The resulting energy plot shows:
- Y-axis: Potential energy in kJ/mol
- X-axis: Minimization steps
- Convergence is indicated by plateauing of energy

# Energy Mimization of the target protein system

In [ ]:
#Change directory to this
%cd /content/drive/MyDrive/MD_GROMACS/charmm-gui/gromacs #or your chosen file_path

In [ ]:
%%bash

init=step3_input
mini_prefix=step4.0_minimization
equi_prefix=step4.1_equilibration
prod_prefix=step5_production
prod_step=step5

# Minimization
gmx grompp -f ${mini_prefix}.mdp -o ${mini_prefix}.tpr -c ${init}.gro -r ${init}.gro -p topol.top -n index.ndx -maxwarn -1
gmx mdrun -v -deffnm ${mini_prefix}

#Using energy to extract the potential energy of the system
echo "Potential" > options
echo " " >> options
gmx energy -f ${mini_prefix}.edr -o em_potential.xvg -xvg none < options

In [ ]:
# Check if file exists
import os
print("File exists:", os.path.exists('em_potential.xvg'))

# If file exists, show first few lines
if os.path.exists('em_potential.xvg'):
    with open('em_potential.xvg', 'r') as f:
        print("\nFirst few lines of the file:")
        print(''.join(f.readlines()[:5]))

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np

# Reading the data
data = np.loadtxt('em_potential.xvg')

# Create plot with basic formatting
plt.figure(figsize=(10, 6))
plt.title('Potential Energy during Minimization', fontsize=12)
plt.xlabel('Energy Minimization Step', fontsize=10)
plt.ylabel(r'E$_P$ [kJ•mol$^{-1}$]', fontsize=10)
plt.grid(True, alpha=0.3)

# Plot the data
plt.plot(data[:,0], data[:,1], color='red', linewidth=2)

# Add minor gridlines
plt.minorticks_on()
plt.grid(which='minor', alpha=0.1)

# Improve layout
plt.tight_layout()

# Show plot
plt.show()

# System Equilibration in Molecular Dynamics

## Theory
Equilibration is performed after energy minimization to:
1. Bring the system to the desired temperature (NVT ensemble)
2. Stabilize system properties before production runs
3. Ensure proper density and pressure conditions

## Process Steps
1. **NVT Equilibration (Canonical Ensemble)**:
   - Number of particles (N), Volume (V), and Temperature (T) are kept constant
   - Uses temperature coupling (thermostat)
   - Important for achieving target temperature
   
2. **Temperature Analysis**:
   - Temperature fluctuations should be stable around target value
   - Standard deviation indicates temperature control quality
   - System should maintain consistent average temperature

## Input Files
1. **step4.1_equilibration.mdp**: Contains equilibration parameters:
   - Temperature coupling parameters
   - Integration timestep
   - Equilibration duration
   - Constraint algorithms

2. **Previous Stage Files**:
   - Minimized structure (step4.0_minimization.gro)
   - Original reference structure (step3_input.gro)
   - System topology (topol.top)
   - Index file (index.ndx)

## Output Analysis
The temperature plot shows:
- System temperature evolution over time
- Temperature stability assessment
- Fluctuations around target temperature

# Equilibration

In [ ]:
!pwd /content/drive/MyDrive/MD_GROMACS/charmm-gui/gromacs #or your chosen file_path

In [ ]:
%%bash
init=step3_input
mini_prefix=step4.0_minimization
equi_prefix=step4.1_equilibration
prod_prefix=step5_production
prod_step=step5

# Equilibration
gmx grompp -f ${equi_prefix}.mdp -o ${equi_prefix}.tpr -c ${mini_prefix}.gro -r ${init}.gro -p topol.top -n index.ndx
gmx mdrun -v -deffnm ${equi_prefix}

#This is a trick to provide interactive options to gmx
!echo "Temperature" > options
!echo " " >> options

#Using energy to extract the temperature of the system during the NVT equil MD
echo "Temperature" > options
echo " " >> options
gmx energy -f ${equi_prefix}.edr -o nvt_temp.xvg -xvg none < options

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np

# Set figure size
plt.figure(figsize=(10, 6))

# Reading the text file containing this information
data = np.loadtxt('nvt_temp.xvg')

# Create the plot
plt.title('Temperature during 0.1 ns Equilibration (NVT)', fontsize=12)
plt.xlabel('Time (ps)', fontsize=10)
plt.ylabel('Temperature [K]', fontsize=10)
plt.grid(True, alpha=0.3)
plt.plot(data[:,0], data[:,1], color='red', linewidth=2)

# Add minor gridlines
plt.minorticks_on()
plt.grid(which='minor', alpha=0.1)

# Improve layout
plt.tight_layout()

# Show the plot
plt.show()

# Print some basic statistics
print("\nTemperature Statistics:")
print(f"Average Temperature: {np.mean(data[:,1]):.2f} K")
print(f"Standard Deviation: {np.std(data[:,1]):.2f} K")
print(f"Min Temperature: {np.min(data[:,1]):.2f} K")
print(f"Max Temperature: {np.max(data[:,1]):.2f} K")

# Production Molecular Dynamics Simulation

## Theory
Production MD is the final phase where the actual simulation data is collected for analysis. This phase:
1. Generates the trajectory for analysis
2. Runs the system under constant conditions (NPT or NVT ensemble)
3. Uses GPU acceleration for efficiency

## Key Components
1. **Simulation Length**:
   - Divided into 10 consecutive runs
   - Helps manage computational resources
   - Allows for checkpoint saves
   - Easier recovery if simulation crashes

2. **GPU Acceleration**:
   - Uses `-nb gpu` flag for non-bonded calculations
   - Significantly speeds up calculations
   - Essential for large systems

3. **Checkpoint Files**:
   - `.cpt` files store simulation state
   - Enable continuous runs
   - Used for simulation recovery
   - Maintain simulation continuity

## Input Requirements
1. **step5_production.mdp**: Production run parameters
   - Timestep
   - Output frequency
   - Ensemble conditions
   - Temperature and pressure coupling

2. **Previous Stage Files**:
   - Equilibrated structure
   - Topology
   - Index file
   - Checkpoint files (after first run)

## Process Flow
1. First run (cnt=1):
   - Starts from equilibration output
   - No checkpoint file needed
2. Subsequent runs (cnt>1):
   - Use previous run's output
   - Include checkpoint files
   - Continue simulation seamlessly

# MDS Production

In [ ]:
!pwd /content/drive/MyDrive/MD_GROMACS/charmm-gui/gromacs #or your chosen file_path

In [ ]:
#%%bash
init=step3_input
mini_prefix=step4.0_minimization
equi_prefix=step4.1_equilibration
prod_prefix=step5_production
prod_step=step5


# Production
cnt=1
cntmax=10
pcnt=0

while (( ${cnt} <= ${cntmax} )); do
    istep=${prod_step}_${cnt}
    pstep=${prod_step}_${pcnt}

	if (( ${cnt} == 1 )); then
        pstep=${equi_prefix}
        gmx grompp -f ${prod_prefix}.mdp -o ${istep}.tpr -c ${pstep}.gro -p topol.top -n index.ndx
	else
        gmx grompp -f ${prod_prefix}.mdp -o ${istep}.tpr -c ${pstep}.gro -t ${pstep}.cpt -p topol.top -n index.ndx
	fi
	gmx mdrun -v -deffnm ${istep} -nb gpu
	((cnt++))
 	((pcnt++))
done

# Trajectory Visualization and Animation

## Theory
After production MD, visualizing the trajectory helps to:
1. Analyze protein dynamics
2. Identify conformational changes
3. Create visual presentations of the simulation

## Trajectory Processing
1. **PBC (Periodic Boundary Conditions) Treatment**:
   - `-pbc mol` option removes periodic boundary effects
   - Centers the molecule in the box
   - Makes visualization more intuitive
   - Prevents protein "jumping" across box boundaries

2. **Output Formats**:
   - **PDB file**: Static structure file
     - Contains atomic coordinates
     - Used as reference structure
   - **XTC file**: Trajectory file
     - Contains dynamic information
     - More compact than full trajectory
     - Compatible with visualization tools

3. **MDsrv Visualization**
   - Web-based molecular viewer
   - Supports trajectory visualization
   - Interactive manipulation
   - No software installation needed

## File Usage
1. Structure file (step5_production_protPBC.pdb):
   - Reference structure
   - Contains protein information
   - Starting point for visualization

2. Trajectory file (step5_production_protPBC.xtc):
   - Contains motion information
   - Time evolution of structure
   - Attached to reference structure for playback

# Animation

In [ ]:
#Using trjconv to extract only the protein atoms from the simulation trajectory
!echo "Protein" > options
!echo "Protein" >> options
!echo " "
!gmx trjconv -s step5_production.tpr -f step5_production.xtc -o step5_production_protPBC.pdb -pbc mol -center < options
!gmx trjconv -s step5_production.tpr -f step5_production.xtc -o step5_production_protPBC.xtc -pbc mol -center < options

## Visualization Instructions
1. Go to MDsrv: https://proteinformatics.uni-leipzig.de/mdsrv.html
2. Follow these steps:
   - Click File > Open
   - Select step5_production_protPBC.pdb
   - Attach step5_production_protPBC.xtc trajectory file
   - Use play button to start animation

## Tips for Visualization
1. Use mouse to rotate and zoom
2. Adjust playback speed as needed
3. Can focus on specific regions of interest
4. Various representation styles available (cartoon, surface, etc.)

# Analysis of MD Simulation Results

## Theory
RMSD (Root Mean Square Deviation) Analysis helps to:
1. Assess structural stability
2. Measure conformational changes
3. Validate simulation quality

## Key Metrics
1. **RMSD Calculation**:
   - Measures structural deviation over time
   - Calculated against:
     - Initial structure
     - Crystal structure (if available)
   - Units: nanometers (nm)

2. **Interpretation Guidelines**:
   - Low RMSD (~0.1-0.3 nm): Structure is stable
   - High RMSD (>0.5 nm): Significant conformational changes
   - Plateau: System has reached equilibrium
   - Continuous increase: Possible instability

3. **Time Units**:
   - Converted to nanoseconds (-tu ns)
   - Helps in better understanding of timescales
   - Standard unit for MD analysis

## Analysis Components
1. **Trajectory Processing**:
   - PBC correction
   - Center of mass motion removal
   - Protein-only analysis

2. **RMSD Calculations**:
   - Against initial structure
   - Against crystal structure (if available)
   - Time evolution analysis

#Result

In [ ]:
%cd /content/drive/MyDrive/MD_GROMACS... #or your chosen file_path

In [ ]:
!gmx trjconv -s step5_production.tpr -f step5_production.xtc -o step5_production_protPBC_output.xtc -pbc mol -center

In [ ]:
!gmx rms -s step5_production.tpr -f step5_production_protPBC_output.xtc -o rmsd.xvg -tu ns

In [ ]:
!gmx rms -s step5_production.tpr -f step5_production_protPBC_output.xtc -o rmsd_xtal.xvg -tu ns

In [ ]:
#Plotting the potential energy of the system
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

#Reading the text file containing this information
data = np.loadtxt('rmsdxtal.xvg')

plt.title('RMSD')
plt.xlabel('Time(ns)')
plt.ylabel(r'RMSD (nm)')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()